In [52]:
import numpy as np
import pandas as pd
import os
import nltk
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import kagglehub

/Users/sudipkhadka/Desktop/Recomendation-System/.rec/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
path = kagglehub.dataset_download("asaniczka/tmdb-movies-dataset-2023-930k-movies")

In [4]:
print("Path to dataset files:", path)

Path to dataset files: /Users/sudipkhadka/.cache/kagglehub/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies/versions/381


In [5]:
directory_path = "/Users/sudipkhadka/.cache/kagglehub/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies/versions/381"
print(os.listdir(directory_path))

['TMDB_movie_dataset_v11.csv']


In [6]:
path = "/Users/sudipkhadka/.cache/kagglehub/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies/versions/381/TMDB_movie_dataset_v11.csv"
file = pd.read_csv(path)

In [7]:
file.shape

(1127268, 24)

In [8]:
file.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')

In [9]:
pd.set_option('display.max_columns', None)

In [10]:
file.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,160000000,https://www.warnerbros.com/movies/inception,tt1375666,en,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,165000000,http://www.interstellarmovie.net/,tt0816692,en,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,185000000,https://www.warnerbros.com/movies/dark-knight/,tt0468569,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,237000000,https://www.avatar.com/movies/avatar,tt0499549,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,220000000,https://www.marvel.com/movies/the-avengers,tt0848228,en,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."


In [11]:
file['id']

0            27205
1           157336
2              155
3            19995
4            24428
            ...   
1127263     728501
1127264     728502
1127265     728503
1127266     728505
1127267    1382000
Name: id, Length: 1127268, dtype: int64

In [12]:
file = file[['id', 'title','overview','tagline', 'genres','keywords']]

In [13]:
file

,id,title,overview,tagline,genres,keywords
0,27205,Inception,"Cobb, a skilled thief who commits corporate es...",Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,The adventures of a group of explorers who mak...,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,Batman raises the stakes in his war on crime. ...,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","future, society, culture clash, space travel, ..."
4,24428,The Avengers,When an unexpected enemy emerges and threatens...,Some assembly required.,"Science Fiction, Action, Adventure","new york city, superhero, shield, based on com..."
...,...,...,...,...,...,...
1127263,728501,California And Its Natural Resources,A look at the vast natural resources in Califo...,NaN,NaN,NaN
1127264,728502,Beep,An old man spends his final days fixing machin...,NaN,Science Fiction,"robot, short film"
1127265,728503,Tribe: The Untold Story of the Making of Vice ...,Two brave young men set out to create the grea...,Don't forget the VENGEANCE,"Comedy, Action, Drama","gun, movie business, writing, satire, tribe, b..."
1127266,728505,Sinkhole,"A young family moves into their dream home, ex...",NaN,Thriller,NaN


In [14]:
file.isna().sum()

id               0
title           13
overview    227262
tagline     969084
genres      450298
keywords    820689
dtype: int64

In [15]:
file.dropna(inplace=True)

In [16]:
file.isna().sum()

id          0
title       0
overview    0
tagline     0
genres      0
keywords    0
dtype: int64

In [17]:
file.duplicated().sum()

np.int64(8)

In [18]:
#display duplicate rows
duplicate_row = file[file.duplicated()]
duplicate_row

,id,title,overview,tagline,genres,keywords
289352,1199990,The Undead,At the end of times a dysfunctional family tak...,The end is just the beginning,"Horror, Thriller","undead, religion, dead children, religious horror"
636555,1192966,Rejoyce!,"After the death of her husband, an elderly vic...",Not your normal Sunday service,"Drama, Comedy","widow, religion, writer, vicar, erotic fantasy..."
636629,1192942,Jamel Herring vs. Nick Molina,Former WBO junior lightweight world champion a...,Heroes On The Hudson,Action,boxing
682954,1260756,Remember,This short documentary produced by the Univers...,"“If you don’t know, why don’t you know?” – Chi...",Documentary,"portland, oregon, world war ii, japanese ameri..."
687723,1261223,A New Life,"A Barber--or professional men's hair coach, as...","A barber--or men's hair coach, as he prefers t...",Documentary,"fashion, life changing, barber"
693652,1256725,Soda Pop Spencer Storms Atlanta: A January 6th...,When a naive soda influencer/reviewer uncovers...,Maybe the means justify the ends...,Comedy,"satire, mockumentary, road movie, political sa..."
702820,1208789,Amy John & Ju-Jitsu Dave,John is left on the boat to fend for his life ...,She’ll be back in thirty minutes.,"Horror, Comedy","coming of age, zombie apocalypse, living on a ..."
715449,1232080,Nakamori Akina - Live In '87 A HUNDRED Days,Live in '87 A HUNDRED days is the seventh vide...,Nakamori Akina - Live in '87 A HUNDRED days,Music,"concert, japanese music, idol"


In [19]:
file.drop_duplicates(inplace=True)

In [20]:
file.duplicated().sum()

np.int64(0)

In [21]:
file.shape

(74301, 6)

In [22]:
file['overview'] = file['overview'].apply(lambda x:x.split())
file['tagline'] = file['tagline'].apply(lambda x:x.split())
file['genres'] = file['genres'].apply(lambda x:x.split())
file['keywords'] = file['keywords'].apply(lambda x:x.split())

In [23]:
file.head()

,id,title,overview,tagline,genres,keywords
0,27205,Inception,"[Cobb,, a, skilled, thief, who, commits, corpo...","[Your, mind, is, the, scene, of, the, crime.]","[Action,, Science, Fiction,, Adventure]","[rescue,, mission,, dream,, airplane,, paris,,..."
1,157336,Interstellar,"[The, adventures, of, a, group, of, explorers,...","[Mankind, was, born, on, Earth., It, was, neve...","[Adventure,, Drama,, Science, Fiction]","[rescue,, future,, spacecraft,, race, against,..."
2,155,The Dark Knight,"[Batman, raises, the, stakes, in, his, war, on...","[Welcome, to, a, world, without, rules.]","[Drama,, Action,, Crime,, Thriller]","[joker,, sadism,, chaos,, secret, identity,, c..."
3,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Enter, the, world, of, Pandora.]","[Action,, Adventure,, Fantasy,, Science, Fiction]","[future,, society,, culture, clash,, space, tr..."
4,24428,The Avengers,"[When, an, unexpected, enemy, emerges, and, th...","[Some, assembly, required.]","[Science, Fiction,, Action,, Adventure]","[new, york, city,, superhero,, shield,, based,..."


In [24]:
def remove_space(s):
    s1 = []
    for i in s:
        s1.append(i.replace(" ", ""))
    return s1

In [25]:
file['overview'] = file['overview'].apply(remove_space)
file['tagline'] = file['tagline'].apply(remove_space)
file['genres'] = file['genres'].apply(remove_space)
file['keywords'] = file['keywords'].apply(remove_space)

In [26]:
file.head()

,id,title,overview,tagline,genres,keywords
0,27205,Inception,"[Cobb,, a, skilled, thief, who, commits, corpo...","[Your, mind, is, the, scene, of, the, crime.]","[Action,, Science, Fiction,, Adventure]","[rescue,, mission,, dream,, airplane,, paris,,..."
1,157336,Interstellar,"[The, adventures, of, a, group, of, explorers,...","[Mankind, was, born, on, Earth., It, was, neve...","[Adventure,, Drama,, Science, Fiction]","[rescue,, future,, spacecraft,, race, against,..."
2,155,The Dark Knight,"[Batman, raises, the, stakes, in, his, war, on...","[Welcome, to, a, world, without, rules.]","[Drama,, Action,, Crime,, Thriller]","[joker,, sadism,, chaos,, secret, identity,, c..."
3,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Enter, the, world, of, Pandora.]","[Action,, Adventure,, Fantasy,, Science, Fiction]","[future,, society,, culture, clash,, space, tr..."
4,24428,The Avengers,"[When, an, unexpected, enemy, emerges, and, th...","[Some, assembly, required.]","[Science, Fiction,, Action,, Adventure]","[new, york, city,, superhero,, shield,, based,..."


In [27]:
file['tag'] = file['overview'] + file['tagline'] + file['genres'] + file['keywords']

In [28]:
file.head()

,id,title,overview,tagline,genres,keywords,tag
0,27205,Inception,"[Cobb,, a, skilled, thief, who, commits, corpo...","[Your, mind, is, the, scene, of, the, crime.]","[Action,, Science, Fiction,, Adventure]","[rescue,, mission,, dream,, airplane,, paris,,...","[Cobb,, a, skilled, thief, who, commits, corpo..."
1,157336,Interstellar,"[The, adventures, of, a, group, of, explorers,...","[Mankind, was, born, on, Earth., It, was, neve...","[Adventure,, Drama,, Science, Fiction]","[rescue,, future,, spacecraft,, race, against,...","[The, adventures, of, a, group, of, explorers,..."
2,155,The Dark Knight,"[Batman, raises, the, stakes, in, his, war, on...","[Welcome, to, a, world, without, rules.]","[Drama,, Action,, Crime,, Thriller]","[joker,, sadism,, chaos,, secret, identity,, c...","[Batman, raises, the, stakes, in, his, war, on..."
3,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Enter, the, world, of, Pandora.]","[Action,, Adventure,, Fantasy,, Science, Fiction]","[future,, society,, culture, clash,, space, tr...","[In, the, 22nd, century,, a, paraplegic, Marin..."
4,24428,The Avengers,"[When, an, unexpected, enemy, emerges, and, th...","[Some, assembly, required.]","[Science, Fiction,, Action,, Adventure]","[new, york, city,, superhero,, shield,, based,...","[When, an, unexpected, enemy, emerges, and, th..."


In [29]:
new_file = file[['id','title','tag']]

In [30]:
new_file['tag'] = new_file['tag'].apply(lambda x: " ".join(x))

/var/folders/2m/53v2qwy10g564nj5mjr0w0nc0000gn/T/ipykernel_42951/3169046667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_file['tag'] = new_file['tag'].apply(lambda x: " ".join(x))


In [31]:
new_file.head(15)

,id,title,tag
0,27205,Inception,"Cobb, a skilled thief who commits corporate es..."
1,157336,Interstellar,The adventures of a group of explorers who mak...
2,155,The Dark Knight,Batman raises the stakes in his war on crime. ...
3,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
4,24428,The Avengers,When an unexpected enemy emerges and threatens...
5,293660,Deadpool,The origin story of former Special Forces oper...
6,299536,Avengers: Infinity War,As the Avengers and their allies have continue...
7,550,Fight Club,A ticking-time-bomb insomniac and a slippery s...
8,118340,Guardians of the Galaxy,"Light years from Earth, 26 years after being a..."
9,680,Pulp Fiction,"A burger-loving hit man, his philosophical par..."


In [32]:
new_file.shape

(74301, 3)

In [33]:
#reducing the size, due the memory limitation
new_file = new_file[:30000]

In [34]:
new_file.shape

(30000, 3)

In [35]:
new_file.iloc[0]['tag']

'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious. Your mind is the scene of the crime. Action, Science Fiction, Adventure rescue, mission, dream, airplane, paris, france, virtual reality, kidnapping, philosophy, spy, allegory, manipulation, car crash, heist, memory, architecture, los angeles, california, dream world, subconscious'

In [37]:
port = PorterStemmer()

In [38]:
def stemmer(txt):
    t = []
    for i in txt.split():
        t.append(port.stem(i))
    return " ".join(t)


In [39]:
new_file['tag'] = new_file['tag'].apply(stemmer)

In [40]:
new_file['tag'][0]

'cobb, a skill thief who commit corpor espionag by infiltr the subconsci of hi target is offer a chanc to regain hi old life as payment for a task consid to be impossible: "inception", the implant of anoth person\' idea into a target\' subconscious. your mind is the scene of the crime. action, scienc fiction, adventur rescue, mission, dream, airplane, paris, france, virtual reality, kidnapping, philosophy, spy, allegory, manipulation, car crash, heist, memory, architecture, lo angeles, california, dream world, subconsci'

In [42]:
cv = CountVectorizer(max_features=2000, stop_words='english')

In [43]:
vector = cv.fit_transform(new_file['tag']).toarray()

In [44]:
vector[1]

array([0, 0, 0, ..., 0, 0, 0])

what we basically did was, we initialized a CountVectorizer to process text data in the 'tag' column of the new_file DataFrame, restricting the vocabulary to the 2000 most frequent words while ignoring common English stop words. It then fits the vectorizer to the data and transforms it into a dense array format that represents the count of each word in the documents.

In [46]:
sim = cosine_similarity(vector)

In [47]:
def recommend(movie):
    index = new_file[new_file['title'] == movie].index[0]
    distances = sorted(list(enumerate(sim[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:8]:
        print(new_file.iloc[i[0]].title)

In [48]:
recommend('Spider-Man')

Spider-Man 3
Spider-Man: Across the Spider-Verse
The Spectacular Spider-Man Attack of the Lizard
Spider-Man 2
Spider-Man: No Way Home
Supergirl
The Amazing Spider-Man


In [50]:
pickle.dump(new_file, open('/Users/sudipkhadka/Desktop/Recomendation-System/model/embedded_name.pkl', 'wb'))

In [51]:

pickle.dump(sim, open('/Users/sudipkhadka/Desktop/Recomendation-System/model/similarity.pkl','wb'))